In [1]:
%matplotlib inline
import os
from glob import glob
from subprocess import Popen, PIPE
import pandas as pd

# Instacart Market Basket Analysis

In [2]:
# Instacart framed another analytics problem in the space of grocery retail
# https://www.kaggle.com/c/instacart-market-basket-analysis
# They've open sourced 3 million instacart orders, providing grocery transactions
# at another level of granularity
# With historical grocery baskets, we seek to predict which products will be in 
# future baskets given a rich relational dataset

## Inspecting the contest data ...

In [3]:
DATA_DIR = '../data/'
for fl in sorted(glob(DATA_DIR + '*.csv')):
    p = Popen(['head', '{}'.format(fl)], stdin=PIPE, stdout=PIPE, stderr=PIPE)
    a, b = p.communicate()
    print('#'*30 + '  Head on file: {}'.format(fl))
    for ll in a.decode('ascii').split('\n'):
        print(ll)

##############################  Head on file: ../data/aisles.csv
aisle_id,aisle
1,prepared soups salads
2,specialty cheeses
3,energy granola bars
4,instant foods
5,marinades meat preparation
6,other
7,packaged meat
8,bakery desserts
9,pasta sauce

##############################  Head on file: ../data/departments.csv
department_id,department
1,frozen
2,other
3,bakery
4,produce
5,alcohol
6,international
7,beverages
8,pets
9,dry goods pasta

##############################  Head on file: ../data/order_products__prior.csv
order_id,product_id,add_to_cart_order,reordered
2,33120,1,1
2,28985,2,1
2,9327,3,0
2,45918,4,1
2,30035,5,0
2,17794,6,1
2,40141,7,1
2,1819,8,1
2,43668,9,0

##############################  Head on file: ../data/order_products__train.csv
order_id,product_id,add_to_cart_order,reordered
1,49302,1,1
1,11109,2,1
1,10246,3,0
1,49683,4,0
1,43633,5,1
1,13176,6,0
1,47209,7,0
1,22035,8,1
36,39612,1,0

##############################  Head on file: ../data/orders.csv
order_id,user_id,ev

In [4]:
# From the sample submission, we see the expected output includes an order_id and a space-separated
# list of product ids. 
# From the description, contestants should "predict which previously purchased 
# products will be in a user’s next order"
# Perhaps the sample examples are globally popular? We will focus on a personalized baseline.
# Submissions are evaluated according to mean F1 score.
# https://www.kaggle.com/c/instacart-market-basket-analysis#evaluation

## Load orders.csv into pandas dataframe

In [5]:
orders = pd.read_csv(DATA_DIR + 'orders.csv')

In [6]:
orders.head(50)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


In [7]:
# Above, we find that orders.csv specifies which instances belong to prior/train/test splits
# The prior split is the largest
# This file also includes day of the week, hour of the day, and days since prior order to
# support an investigation into the relationship between grocery baskets and periodic effects

## Before all that, let's combine orders, users, and products over the prior split

In [8]:
prior_orders = pd.merge(orders.query("eval_set=='prior'")[['order_id', 'user_id']], 
         pd.read_csv(DATA_DIR + 'order_products__prior.csv'))

In [9]:
prior_orders.head(30)

,order_id,user_id,product_id,add_to_cart_order,reordered
0,2539329,1,196,1,0
1,2539329,1,14084,2,0
2,2539329,1,12427,3,0
3,2539329,1,26088,4,0
4,2539329,1,26405,5,0
5,2398795,1,196,1,1
6,2398795,1,10258,2,0
7,2398795,1,12427,3,1
8,2398795,1,13176,4,0
9,2398795,1,26088,5,1


# Creature-of-Habit Assumption

## People will reorder products they've often reordered in the past

In [10]:
# We will filter the dataframe above for reordered products, focusing on user_id & product_id
# To start, let's simply aggregate by the most commonly reordered product per user

In [11]:
usr_prod_mode_srs = prior_orders[['user_id', 'product_id', 'reordered']].query("reordered=='1'").drop('reordered', axis=1).groupby(['user_id']).agg(lambda x:x.value_counts().index[0])

In [12]:
order_user_srs = orders.query("eval_set=='test'")[['order_id', 'user_id']]

In [13]:
order_product_df = pd.merge(usr_prod_mode_srs.reset_index(), order_user_srs)[['order_id', 'product_id']]

In [14]:
order_prod_dict = dict(order_product_df.values)

In [15]:
# Now we have a dictionary to lookup the most commonly reordered product over the train split for each user

In [16]:
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [17]:
sub['products'] = sub['order_id'].replace(order_prod_dict)

In [18]:
sub.head()

,order_id,products
0,17,13107
1,34,39180
2,137,41787
3,182,9337
4,257,49235


### This model assumes:
*  Ignores changing preference in time
*  Assumes each order comes with only one item
*  Ignores similarity between users and items

### Further Directions
*  Explore similarity among items, users clustering/embeddings
*  Explore temporal trends
*  Modeling the user's number of reorders: 0, 1, 2, ...
*  Optimization with gradient boosting: http://blog.kaggle.com/2017/09/21/instacart-market-basket-analysis-winners-interview-2nd-place-kazuki-onodera/